<a href="https://colab.research.google.com/github/shakir507/BacklogModels/blob/main/PhaseSpaceExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code investigates the parameter space of the backlog-resource model for healthcare, exploring work hours and the amplitude of demand surges to determine the resources required. A surge is assumed to occur against the base demand for a time period 'P', after which demand returns to the base level.